In [5]:
import nltk
import re
import csv
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import datefinder
from datetime import datetime

class experience_details():
    
    def search_keyword(self,text, keyword_list):
        for word in keyword_list:
            word = str(word)
            if word.title() in text or word.upper() in text or word.capitalize() in text:
                return True
        return False
    
    def load_work_segment(self,pdf_to_text_list,education_keywords,education_degree_category,
                               project_keywords,skill_keywords, work_experience_keywords,other_keywords,
                              work_segment):
        
        #print("===Insert text list")
        #print(len(pdf_to_text_list))
        
        text_list = pdf_to_text_list
        def validate_text(text):
            
            user_segment = []

            # check email in name object
            if text.find("@") > 0 or text.find("github.com") >= 0 or text.find("linkedin.com") >= 0:
                user_segment.append(text)
                print ("Email found")
                return False

            # check phone number in name object
            cell = re.search("\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4}",
                             text)
            if cell:
                print ("Cell found")
                user_segment.append(text)
                return False

            # check hyperlinks in name object
            urls = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', text)
            if len(urls) > 0:
                print ("link found")
                user_segment.append(text)
                return False

            if text.find("GPA") > 0:
                print ("GPA found")
                education_segment.append(text)
                return False

            return True
        
        #print("===== before going in to loop list======")
        #print(len(text_list))
        # Extract Work Segment
        #for i, text in enumerate(pdf_to_text_list):
        for i,text in enumerate(text_list):
            #print(i,"=====",text)
            flag = False
            if self.search_keyword(text, work_experience_keywords):
                #print(text)
                work_segment.append(text)
                #print('Segment' , work_segment)
                #print('bhauik')
                #print(i)
                i += 1
                #print("Next Line",i)
                flag = True
                try:
                    while True:
                        text = text_list[i]
                        #print("while loop",text)
                        #print("line number",i)
                        if not self.search_keyword(text, education_keywords) and not self.search_keyword(text,
                                                                                              education_degree_category) and not self.search_keyword(
                                text, project_keywords) and not self.search_keyword(text, skill_keywords) and not self.search_keyword(
                                text, other_keywords):

                            #if validate_text(text):
                            work_segment.append(text)
                        else:
                            break
                        i += 1
                except:
                    print("last line")
            if flag:
                break
                
        #print(work_segment)
        return work_segment
    
    
    "===== extract work experience ============"
    
    def extract_work_experience(self,work_experience_keywords,company_names,
                                job_title_names,work_seg,work_experience,infoDict):
        
        work_segment = work_seg
        #print("=====",work_segment)
        work_experience = work_experience
        infoDict = infoDict
        infoDict['work_keyword'] = []
        
        noun_phrases = []
        closest_company_1, closest_company_2 = "", ""
        comp = []
        pos_tit = []
        work_experience['Date'] = []
        work_experience['Job_Title'] = []
        work_experience['company'] = []
        max_score, longest_length = 0, 0

        try:
            for text in (work_segment):
                lines = nltk.word_tokenize(text)
                tags = nltk.pos_tag(lines)

                nouns = [word for word,pos in tags if(pos == "NN")]
                company = " ".join(nouns)
                comp.append(company)


                title = [word for word,pos in tags if(pos == "NNP")]
                ti = " ".join(title)
                pos_tit.append(ti)

            for i in range(0,len(work_segment)):

                closest_title = ''
                closest_company = ''
                ty = {}

                title_flag = False
                company_flag = False
                date_flag = False
                #print(len(work_segment))
                text = work_segment[i]
                #print(text)
                #print(i)
                if not title_flag:
                    for title in job_title_names:
                        if type(title) == str:
                            title = title
                        else:
                            break

                        if title in text:
                            #print(text)
                            if (re.match('[?.!]$', title) is None):
                                closest_title = title
                            #print(closest_title)
                                title_flag = True



                        score = fuzz.ratio(str(title).lower(), text.lower())
                        #print (university, score)
                        if score > 90 and max_score < score:
                            max_score = score
                            if (re.match('[?.!]$', title) is None):
                                closest_title = title
                                title_flag = True
                            #print(closest_title)

                    if title_flag:
                        work_experience['Job_Title'].append(closest_title)

                        #print("=== IF title fetch ======", work_experience['Job_Title'])
                        #print(i)
                        j = i
                        #print(j)

                        while (True):
                            if not company_flag:

                                for company in company_names:
                                    #print(company)
                                    #print(text)
                                    if (company.lower()) in (text.lower()) and longest_length <= len(company) and text != "":
                                        #print(text, '=== Parse')
                                        #print(i)
                                        #print(str(company))
                                        longest_length = len(company)
                                        closest_company_1 = company
                                        #print(closest_company_1,"==== 1")
                                        company_flag = True
                                    #elif not closest_company_1:


                                    score = fuzz.ratio(str(company).lower(), text.lower())
                                    #print (university, score)
                                    if score > 90 and max_score < score and not text == " ":
                                        #print(company,score)
                                        max_score = score
                                        closest_company_2 = company
                                        company_flag = True
                                        #print(closest_company_2,"===2")

                                if company_flag:
                                    if len(closest_company_1) > len(closest_company_2):
                                        closest_company = closest_company_1
                                    else:
                                        closest_company = closest_company_2

                                    if closest_company:
                                        #print(closest_company)
                                        work_experience['company'].append(closest_company)


                            if not date_flag:
                                #print("==== again date ====", date_flag)
                                matches = re.findall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*(?:-|\.|\s|,)\s?\d{,2}[a-z]*(?:-|,|\s)?\s?\d{2,4})',text)
                                #print(matches)
                                if matches:
                                    #print(len(matches))
                                    if len(matches) == 1:
                                        ty = {'start_year':matches[0],'end_year':"Present"}
                                    else:
                                        ty = {'start_year':matches[0],'end_year':matches[1]}
                                    work_experience['Date'].append(ty)
                                    date_flag = True
                                else:
                                    matches = re.findall(r'\b\d{4}\b',text)
                                    if matches:
                                        if len(matches) == 1:
                                            ty = {'start_year':matches[0],'end_year':"Present"}
                                        else:
                                            ty = {'start_year':matches[0],'end_year':matches[1]}

                                        work_experience['Date'].append(ty)
                                        date_flag = True

                            break

                            i+=1

                            if not i >= len(work_segment):
                                text = work_segment[i]
                            else:
                                print("Check in next lines")
                                k = 0
                                print("while loop === ",j)

                                while (k<j):
                                    text = work_segment[k]
                                    #print(k,"second while loop=========", text)

                                    if not work_experience['company']:
                                        for company in company_names:
                                            #print(company)
                                            #print(text)
                                            if (company.lower()) in (text.lower()) and longest_length <= len(company) and text != "":
                                                #print(text, '=== Parse')
                                                #print(i)
                                                #print(str(company))
                                                longest_length = len(company)
                                                closest_company_1 = company
                                                #print(closest_company_1,"==== 1")
                                                company_flag = True
                                            #elif not closest_company_1:


                                            score = fuzz.ratio(str(company).lower(), text.lower())
                                            #print (university, score)
                                            if score > 90 and max_score < score and not text == " ":
                                                #print(company,score)
                                                max_score = score
                                                closest_company_2 = company
                                                company_flag = True
                                                #print(closest_company_2,"===2")

                                        if company_flag:
                                            if len(closest_company_1) > len(closest_company_2):
                                                closest_company = closest_company_1
                                            else:
                                                closest_company = closest_company_2

                                            if closest_company:
                                                print(closest_company)
                                                work_experience['company'].append(closest_company)
                                                company_flag = True

                                    if not work_experience['Date']:
                                        matches = re.findall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*(?:-|\.|\s|,)\s?\d{,2}[a-z]*(?:-|,|\s)?\s?\d{2,4})',text)
                                        #print(matches)
                                        if matches:
                                            #print(len(matches))
                                            if len(matches) == 1:
                                                ty = {'start_year':matches[0],'end_year':"Present"}
                                            else:
                                                ty = {'start_year':matches[0],'end_year':matches[1]}
                                            work_experience['Date'].append(ty)
                                            date_flag = True
                                        else:
                                            matches = re.findall(r'\b\d{4}\b',text)
                                            if matches:
                                                if len(matches) == 1:
                                                    ty = {'start_year':matches[0],'end_year':"Present"}
                                                else:
                                                    ty = {'start_year':matches[0],'end_year':matches[1]}

                                                work_experience['Date'].append(ty)
                                                date_flag = True
                                    #print("====== loop ======", work_experience['company'])
                                    #print("======////////==========")
                                    #print("======= loop ========", work_experience['Date'])
                                    k+=1
                                break

                #print(i,"===== sentance finish =========")
                if not closest_title or not closest_company or not ty :
                    #print(text, "========not parse")
                    infoDict['work_keyword'].append(text)
            e = work_experience
            return e,infoDict

        except IndexError:
            return print('Index Error')
        
    "============== extract work experience ============"
    
    def extract_work_experience(self,work_experience_keywords,company_names,
                                job_title_names,work_seg,work_experience,infoDict):
        
        work_segment = work_seg
        #print("=====",work_segment)
        work_experience = work_experience
        infoDict = infoDict
        infoDict['work_keyword'] = []

        closest_company_1, closest_company_2 = "", ""
        comp = []
        pos_tit = []
        work_experience['Date'] = []
        work_experience['Job_Title'] = []
        work_experience['company'] = []
        #work_experience['keywords'] = []
        max_score, longest_length = 0, 0

        for text in (work_segment):
            lines = nltk.word_tokenize(text)
            tags = nltk.pos_tag(lines)

            nouns = [word for word,pos in tags if(pos == "NN")]
            company = " ".join(nouns)
            comp.append(company)


            title = [word for word,pos in tags if(pos == "NNP")]
            ti = " ".join(title)
            pos_tit.append(ti)

        for i in range(0,len(work_segment)):

            closest_title = ''
            closest_company = ''
            ty = {}

            title_flag = False
            company_flag = False
            date_flag = False
            #print(len(work_segment))
            text = work_segment[i]
            #print(text)
            #print(i)
            if not company_flag:
                for company in company_names:
                    #print(company)
                    #print(text)
                    if (company.lower()) in (text.lower()) and longest_length <= len(company):
                        #print(text, '=== Parse')
                        #print(i)
                        #print(str(company))
                        if re.match(r'[a-zA-Z0-9_].',company) is not None:
                            longest_length = len(company)
                            closest_company_1 = company
                            #print(closest_company_1,"==== 1")
                            company_flag = True
                    #elif not closest_company_1:


                    score = fuzz.ratio(str(company).lower(), text.lower())
                    #print (university, score)
                    if score > 90 and max_score < score and not text == " ":
                        #print(company,score)
                        max_score = score
                        if re.match(r'[a-zA-Z0-9_].',company) is not None:
                            closest_company_2 = company
                            company_flag = True
                        #print(closest_company_2,"===2")

                if company_flag:
                    if len(closest_company_1) > len(closest_company_2):
                        closest_company = closest_company_1
                    else:
                        closest_company = closest_company_2

                    if closest_company:
                        print(closest_company)
                        work_experience['company'].append(closest_company)

                    #print(i)
                    j = i
                    #print(j)

                    while (True):
                        if not title_flag:
                            for title in job_title_names:
                                if type(title) == str:
                                    title = title
                                else:
                                    break

                                if title in text:
                                    #print(text)
                                    if re.match(r'[a-zA-Z0-9_].',title) is not None:
                                        closest_title = title
                                        #print(closest_title)
                                        title_flag = True



                                score = fuzz.ratio(str(title).lower(), text.lower())
                                #print (university, score)
                                if score > 90 and max_score < score:
                                    max_score = score
                                    if re.match(r'[a-zA-Z0-9_].',title) is not None:
                                        closest_title = title
                                        title_flag = True
                                    #print(closest_title)

                            if title_flag:
                                work_experience['Job_Title'].append(closest_title)

                        if not date_flag:
                            matches = re.findall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*(?:-|\.|\s|,)\s?\d{,2}[a-z]*(?:-|,|\s)?\s?\d{2,4})',text)
                            #print(matches)
                            if matches:
                                #print(len(matches))
                                if len(matches) == 1:
                                    ty = {'start_year':matches[0],'end_year':"Present"}
                                else:
                                    ty = {'start_year':matches[0],'end_year':matches[1]}
                                work_experience['Date'].append(ty)
                                date_flag = True
                            else:
                                matches = re.findall(r'\b\d{4}\b',text)
                                if matches:
                                    if len(matches) == 1:
                                        ty = {'start_year':matches[0],'end_year':"Present"}
                                    else:
                                        ty = {'start_year':matches[0],'end_year':matches[1]}

                                    work_experience['Date'].append(ty)
                                    date_flag = True
                        i+=1

                        if not i >= len(work_segment):
                            text = work_segment[i]
                        else:
                            #print("Check in next lines")
                            k = 0
                            #print("while loop === ",j)

                            while (k<j):
                                text = work_segment[k]
                                #print(k,"second while loop=========", text)

                                if not work_experience['Job_Title']:
                                    for title in job_title_names:
                                        if type(title) == str:
                                            title = title
                                        else:
                                            break

                                        if title in text:
                                            #print(text)
                                            if re.match(r'[a-zA-Z0-9_].',title) is not None:
                                                closest_title = title
                                                #print(closest_title)
                                                title_flag = True

                                        score = fuzz.ratio(str(title).lower(), text.lower())
                                        #print (university, score)
                                        if score > 90 and max_score < score:
                                            max_score = score
                                            if re.match(r'[a-zA-Z0-9_].',title) is not None:
                                                closest_title = title
                                                title_flag = True
                                    #print(closest_title)

                                    if title_flag:
                                        work_experience['Job_Title'].append(closest_title)

                                if not work_experience['Date']:
                                    matches = re.findall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*(?:-|\.|\s|,)\s?\d{,2}[a-z]*(?:-|,|\s)?\s?\d{2,4})',text)
                                    #print(matches)
                                    if matches:
                                        #print(len(matches))
                                        if len(matches) == 1:
                                            ty = {'start_year':matches[0],'end_year':"Present"}
                                        else:
                                            ty = {'start_year':matches[0],'end_year':matches[1]}
                                        work_experience['Date'].append(ty)
                                        date_flag = True
                                    else:
                                        matches = re.findall(r'\b\d{4}\b',text)
                                        if matches:
                                            if len(matches) == 1:
                                                ty = {'start_year':matches[0],'end_year':"Present"}
                                            else:
                                                ty = {'start_year':matches[0],'end_year':matches[1]}

                                            work_experience['Date'].append(ty)
                                            date_flag = True
                                k+=1
                            break


            if not closest_company or not closest_title or not ty:
                infoDict['work_keyword'].append(text)

        e = work_experience
        #print(e)
        return e, infoDict

        if not work_experience['company']:
            print("********Title First function**********")
            return self.extract_exp_title_first() 

In [6]:
experience_details()